# `sciflow` more detailed ML example - Abalone Dataset

In [ ]:
# default_exp abalone

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# export

import logging

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.addHandler(logging.StreamHandler())

# Import Variables

In [ ]:
# export

import json
import os
import subprocess
import sys
import time
from pathlib import Path

import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.impute import SimpleImputer
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, StandardScaler

In [ ]:
# export

bucket = os.environ["SCIFLOW_BUCKET"]
dataset_key_prefix = "datasets/abalone"
data_path = f"s3://{bucket}/{dataset_key_prefix}/abalone-dataset.csv"
hyperparameters = {"max_iter": 20}
model_dir = "/tmp/replacewithgenerated"

In [ ]:
if not Path(model_dir).exists():
    Path(model_dir).mkdir()

In [ ]:
# exportn_step:preprocess


def preprocess(data_path):
    # Since we get a headerless CSV file we specify the column names here.
    feature_columns_names = [
        "sex",
        "length",
        "diameter",
        "height",
        "whole_weight",
        "shucked_weight",
        "viscera_weight",
        "shell_weight",
    ]
    feature_columns_names = [c.lower() for c in feature_columns_names]

    has_additional_dependencies = Path("requirements.txt").exists()
    if has_additional_dependencies:
        logger.info("Installing additional dependencies from requirements.txt")
        subprocess.check_call(
            [sys.executable, "-m", "pip", "install", "-r", "requirements.txt"]
        )
        logger.debug("Installed additional dependencies")

    df = pd.read_csv(data_path)
    # Convert column names ot lwoer case
    df.columns = [c.strip().lower().replace(" ", "_") for c in df.columns]

    logger.debug("Defining transformers.")
    numeric_features = list(feature_columns_names)
    numeric_features.remove("sex")
    numeric_transformer = Pipeline(
        steps=[
            ("imputer", SimpleImputer(strategy="median")),
            ("scaler", StandardScaler()),
        ]
    )

    categorical_features = ["sex"]
    categorical_transformer = Pipeline(
        steps=[
            (
                "ordinal",
                OrdinalEncoder(
                    handle_unknown="use_encoded_value", unknown_value=np.nan
                ),
            )
        ]
    )
    preprocess = ColumnTransformer(
        transformers=[
            ("num", numeric_transformer, numeric_features),
            ("cat", categorical_transformer, categorical_features),
        ]
    )

    logger.info("Applying transforms.")
    y = df["rings"]
    X_pre = df.drop("rings", axis=1)
    X_pre = preprocess.fit_transform(X_pre)
    y_pre = y.to_numpy().reshape(len(y), 1)

    X = np.concatenate((y_pre, X_pre), axis=1)

    logger.info(
        "Splitting %d rows of data into train, validation, test datasets.", len(X)
    )
    np.random.shuffle(X)
    train, validation, test = np.split(X, [int(0.7 * len(X)), int(0.85 * len(X))])

    categorical_mask = [False] * len(numeric_features) + [True] * len(
        categorical_features
    )
    columns = ["rings"] + numeric_features + categorical_features
    results = {
        "train": pd.DataFrame(data=train, columns=columns),
        "validation": pd.DataFrame(data=validation, columns=columns),
        "test": pd.DataFrame(data=test, columns=columns),
        "categorical_mask": categorical_mask,
    }
    return results

In [ ]:
# export


def plot_perm_importances(model_dir, model, X, y, dataset_name: str):
    result = permutation_importance(
        model, X, y, n_repeats=10, random_state=42, n_jobs=2
    )
    sorted_idx = result.importances_mean.argsort()

    fig, ax = plt.subplots()
    ax.boxplot(
        result.importances[sorted_idx].T, vert=False, labels=X.columns[sorted_idx]
    )
    ax.set_title(f"Permutation Importances: {dataset_name}")
    fig.tight_layout()
    plt.savefig(
        os.path.join(model_dir, f"permutation_importances_{dataset_name}.pdf"),
        dpi=150,
    )

In [ ]:
# exportn_step:fit


def fit(model_dir, train, validation, categorical_mask, hyperparameters):
    model = HistGradientBoostingRegressor(
        **hyperparameters, categorical_features=categorical_mask
    )

    logger.debug(f"Using GBM with {model.max_iter} estimators")

    train_data = train.copy()
    y_train = train_data.iloc[:, 0].to_numpy()
    train_data.drop(train_data.columns[0], axis=1, inplace=True)
    X_train = train_data
    logger.debug(f"Training data loaded")

    st = time.time()
    model.fit(X_train, y_train)
    train_time = (time.time() - st) / 60
    logger.info(f"Model trained after: {train_time} minutes")

    validation_data = validation.copy()
    y_val = validation_data.iloc[:, 0].to_numpy()
    validation_data.drop(validation_data.columns[0], axis=1, inplace=True)
    X_val = validation_data
    logger.debug(f"Validation data loaded")

    y_pred_train = model.predict(X_train)
    y_pred_val = model.predict(X_val)

    logger.debug("Calculating mean squared errors.")
    mse_train = mean_squared_error(y_train, y_pred_train)
    std_train = np.std(y_train - y_pred_train)
    mse_val = mean_squared_error(y_val, y_pred_val)
    std_val = np.std(y_val - y_pred_val)

    logger.debug("Calculated mean squared errors.")
    logger.info(f"Train MSE={mse_train}; Train STD={std_train};")
    logger.info(f"Validation MSE={mse_val}; Validation STD={std_val};")

    plot_perm_importances(model_dir, model, X_train, y_train, "train")
    plot_perm_importances(model_dir, model, X_val, y_val, "validation")

    joblib.dump(model, os.path.join(model_dir, "model.joblib"))
    logger.info("Model persisted to file.")

In [ ]:
# exportn_step:evaluate


def evaluation(model_dir, test):
    has_additional_dependencies = Path("requirements.txt").exists()
    if has_additional_dependencies:
        logger.info("Installing additional dependencies from requirements.txt")
        subprocess.check_call(
            [sys.executable, "-m", "pip", "install", "-r", "requirements.txt"]
        )
        logger.debug("Installed additional dependencies")

    logger.debug("Loading sklearn model.")
    model = joblib.load(f"{model_dir}/model.joblib")

    logger.debug("Reading test data.")
    df = test.copy()
    y_test = df.iloc[:, 0].to_numpy()
    df.drop(df.columns[0], axis=1, inplace=True)
    X_test = df
    logger.debug("Read in test data.")

    logger.info("Performing predictions against test data.")
    predictions = model.predict(X_test)

    logger.debug("Calculating mean squared error.")
    mse = mean_squared_error(y_test, predictions)
    std = np.std(y_test - predictions)
    report_dict = {
        "regression_metrics": {
            "mse": {"value": mse, "standard_deviation": std},
        },
    }

    Path(model_dir).mkdir(parents=True, exist_ok=True)

    plot_perm_importances(model_dir, model, X_test, y_test, "test")

    logger.info("Writing out evaluation report with mse: %f", mse)
    evaluation_path = f"{model_dir}/evaluation.json"
    with open(evaluation_path, "w") as f:
        f.write(json.dumps(report_dict))

    logger.debug("Completed evaluation.")

In [ ]:
# slow

results = preprocess(data_path)

In [ ]:
# slow

fit(
    model_dir,
    results["train"],
    results["validation"],
    results["categorical_mask"],
    hyperparameters,
)

In [ ]:
# slow

evaluation(model_dir, results["test"])